# Boston Housing

In [1]:
include("../src/load.jl")
using AlgorithmicRecourse, MLDatasets, Flux
using Plots, PlotThemes
theme(:juno)
using Logging
disable_logging(Logging.Info)

LogLevel(1)

## Training the classifier

In [2]:
using MLDatasets, Statistics
X = BostonHousing.features()
y = BostonHousing.targets()
y = y .>= median(y); # binary target

To start off we will just train a single neural network for the binary classification task.

In [3]:
# Prepare data and model:
using Random
Random.seed!(1234)
minib_size = 100
data = Flux.Data.DataLoader((X,y),batchsize=minib_size)
minib_actual_sizes = map(mb -> length(mb[2]), data)
nn = Classifiers.build_model(input_dim=size(X)[1], n_hidden=100)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y)

loss (generic function with 1 method)

The model achieves decent training accuracy

In [4]:
# Train model:
using Flux.Optimise: update!, ADAM
using Statistics, StatsBase
opt = ADAM()
epochs = 50
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
accuracy(data) = mean(map(minib -> sum(round.(Flux.σ.(nn(minib[1]))) .== minib[2])/length(minib[2]),data), weights(minib_actual_sizes))

using Plots
anim = Animation()
avg_l = [avg_loss(data)]
p1 = scatter( ylim=(0,avg_l[1]), xlim=(0,epochs), legend=false, xlab="Epoch", title="Average loss")
acc_l = [accuracy(data)]
p2 = scatter( ylim=(0.5,1), xlim=(0,epochs), legend=false, xlab="Epoch", title="Accuracy")

for epoch = 1:epochs
  for d in data
    gs = gradient(params(nn)) do
      l = loss(d...)
    end
    update!(opt, params(nn), gs)
  end
  avg_l = vcat(avg_l,avg_loss(data))
  plot!(p1, [0:epoch], avg_l, color=1)
  scatter!(p1, [0:epoch], avg_l, color=1)
  acc_l = vcat(acc_l,accuracy(data))
  plot!(p2, [0:epoch], acc_l, color=1)
  scatter!(p2, [0:epoch], acc_l, color=1)
  plt=plot(p1,p2, size=(600,300))
  frame(anim, plt)
end

gif(anim, "www/boston_housing_single_nn.gif", fps=10);

![](www/boston_housing_single_nn.gif)

Next we will build and train a deep ensemble.

In [5]:
K = 2
𝓜 = Classifiers.build_ensemble(K,kw=(input_dim=size(X)[1], n_hidden=100));
𝓜, anim = Classifiers.forward(𝓜, data, opt, n_epochs=30, plot_every=10); # fit the ensemble
gif(anim, "www/boston_housing_ensemble_loss.gif", fps=25);

![](www/boston_housing_ensemble_loss.gif)

In [6]:
using Random
Random.seed!(1234)
x̅ = X[:,(y.==0)'][:,rand(1:length((y.==0)'))] # select individual sample
x̅ = reshape(x̅, (length(x̅),1))
γ = 0.75
target=1.0
𝑴=Classifiers.FittedEnsemble(𝓜);

In [7]:
𝓜[1](x̅)

1×1 Matrix{Float64}:
 -3.90562567615722

In [8]:
generator = GreedyGenerator(0.1,20,:logitbinarycrossentropy,nothing)
recourse = generate_recourse(generator, x̅, 𝑴, target, γ, T=100); # generate recourse

In [9]:
𝓜[1](x̅)

1×1 Matrix{Float64}:
 NaN